In [ ]:
#Задача 1 Права администратора

In [1]:
# Глобальная переменная для хранения текущей роли пользователя
user_role = 'user'

def role_required(role: str):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user_role == role:
                return func(*args, **kwargs)
            else:
                return 'Permission denied'
        return wrapper
    return decorator

@role_required('admin')
def secret_resource() -> str:
    return 'Permission accepted'

# Примеры использования
user_role = 'admin'
print(secret_resource())  # Ожидаемый вывод: 'Permission accepted'

user_role = 'manager'
print(secret_resource())  # Ожидаемый вывод: 'Permission denied'


Permission accepted
Permission denied


In [ ]:
#Задача 2 Кэширование

In [2]:
from functools import wraps

def cache(db: str, expiration: int):
    cache_storage = {}
    def decorator(func):
        @wraps(func)
        def wrapper(thing):
            if thing in cache_storage and cache_storage[thing]['expire'] > 0:
                cache_storage[thing]['expire'] -= 1
                return f"Info about: {thing} cached in {db}, expire={cache_storage[thing]['expire']}"
            else:
                cache_storage[thing] = {'result': func(thing), 'expire': expiration}
                return f"Info about: {thing} from {db}, now cached with expire={expiration}"
        return wrapper
    return decorator

@cache('postgresql', 5)
def get_info(thing: str) -> str:
    return f"Info about: {thing}"

# Примеры использования
print(get_info('bike_store'))
print(get_info('bike_store'))
print(get_info('bike_store'))
print(get_info('bike_store'))
print(get_info('bike_store'))
print(get_info('bike_store'))


Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in postgresql, expire=3
Info about: bike_store cached in postgresql, expire=2
Info about: bike_store cached in postgresql, expire=1
Info about: bike_store cached in postgresql, expire=0


In [ ]:
#Задача 3 Контекстный менеджер safe_write

In [17]:
import locale
print(locale.getpreferredencoding())

cp1251


In [18]:
import os
import shutil

class safe_write:
    def __init__(self, filename):
        self.filename = filename
        self.temp_filename = filename + ".tmp"

    def __enter__(self):
        self.backup_exists = os.path.exists(self.filename)
        if self.backup_exists:
            shutil.copy(self.filename, self.temp_filename)
        self.file = open(self.filename, 'w', encoding='utf-8')
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        self.file.close()
        if exc_type is not None:
            print(f"Во время записи в файл было возбуждено исключение {exc_type.__name__}")
            if self.backup_exists:
                shutil.copy(self.temp_filename, self.filename)
                os.remove(self.temp_filename)
            else:
                os.remove(self.filename)
            return True
        if self.backup_exists:
            os.remove(self.temp_filename)

# Примеры работы
with safe_write('undertale.txt') as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого.')

with open('undertale.txt', encoding='utf-8') as file:
    print(file.read())

with safe_write('under_tale.txt') as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого. \n')

try:
    with safe_write('under_tale.txt') as file:
        print('Если ты будешь любознательным, то будешь много знающим.', file=file, flush=True)
        raise ValueError
except ValueError:
    pass

with open('under_tale.txt', encoding='utf-8') as file:
    print(file.read())


Я знаю, что ничего не знаю, но другие не знают и этого.
Во время записи в файл было возбуждено исключение ValueError
Я знаю, что ничего не знаю, но другие не знают и этого. 

